DATA COLLECTION & UNDERSTANDING

In [6]:
import pandas as pd
import numpy as np



In [8]:
url = "https://raw.githubusercontent.com/AlvitoDwiP/churn/main/Dataset_Example/churn/WA_Fn-UseC_-Telco-Customer-Churn.csv"
df = pd.read_csv(url)
df.head()



,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [10]:
df.shape

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


Yang normal:
- tenure → int
- MonthlyCharges → float

Yang bermasalah:
- TotalCharges → object ❌

Kenapa bermasalah? Object = teks jadinya tidak bisa dihitung, dibandingan dan tidak bisa pakai value




In [11]:
df.isnull().sum()


,0
customerID,0
gender,0
SeniorCitizen,0
Partner,0
Dependents,0
tenure,0
PhoneService,0
MultipleLines,0
InternetService,0
OnlineSecurity,0


In [15]:
df["customerID"].nunique()

df.shape[0]

7043

Identitas:
- customerID
- Target:
- Churn

Numerik:
- tenure
- MonthlyCharges
- TotalCharges (kotor)

Kategori:
- gender
- Contract
- PaymentMethod
- InternetService
- TechSupport
- dll

JADI OUTPUT YANG BISA DIDAPAT DARI ORIENTASI DATA TERSEBUT ADALAH :
- Dataset punya 7043 pelanggan dan 21 kolom
- Target ada di Churn
- TotalCharges tersimpan sebagai teks (object), sehingga tidak bisa dilakukan operasi numerik seperti perbandingan, agregasi, atau dipakai sebagai proxy nilai pelanggan sebelum dibersihkan dan dikonversi ke tipe numerik.
- customerID tidak mengandung informasi perilaku atau nilai pelanggan, sehingga tidak memiliki daya prediktif terhadap churn dan justru berisiko menyebabkan model menghafal identitas, bukan pola.
- Mayoritas fitur adalah kategori

Kolom yang paling berpotensi terkait churn adalah MonthlyCharges dan tenure.
karena:

- tenure: pelanggan baru belum loyal → churn lebih tinggi
- MonthlyCharges: biaya tinggi → sensitivitas harga → churn

PREPROCESSING DATA

In [16]:
df["TotalCharges"].unique()[:10]


array(['29.85', '1889.5', '108.15', '1840.75', '151.65', '820.5',
       '1949.4', '301.9', '3046.05', '3487.95'], dtype=object)

In [20]:
df["TotalCharges"] = df["TotalCharges"].replace(" ", np.nan)
df["TotalCharges"] = pd.to_numeric(df["TotalCharges"])
df["TotalCharges"].isnull().sum()

np.int64(11)

karena masih terdapat NaN, kita putuskan untuk isi dengan logika bisnis (MonthlyCharges × tenure)

In [22]:
mask = df["TotalCharges"].isnull()
df.loc[mask, "TotalCharges"] = df.loc[mask, "MonthlyCharges"] * df.loc[mask, "tenure"]

df[["tenure", "MonthlyCharges", "TotalCharges"]].describe()


,tenure,MonthlyCharges,TotalCharges
count,7043.000000,7043.000000,7043.000000
mean,32.371149,64.761692,2279.734304
std,24.559481,30.090047,2266.794470
min,0.000000,18.250000,0.000000
25%,9.000000,35.500000,398.550000
50%,29.000000,70.350000,1394.550000
75%,55.000000,89.850000,3786.600000
max,72.000000,118.750000,8684.800000


- Tidak ada TotalCharges negatif
- Mean masuk akal
- Max tidak absurd

In [23]:
(df["TotalCharges"] < df["MonthlyCharges"]).sum()


np.int64(11)

In [24]:
df[df["TotalCharges"] < df["MonthlyCharges"]][["tenure","MonthlyCharges","TotalCharges"]].head(20)


,tenure,MonthlyCharges,TotalCharges
488,0,52.55,0.0
753,0,20.25,0.0
936,0,80.85,0.0
1082,0,25.75,0.0
1340,0,56.05,0.0
3331,0,19.85,0.0
3826,0,25.35,0.0
4380,0,20.00,0.0
5218,0,19.70,0.0
6670,0,73.35,0.0


tenure kecil (0 atau 1). Kalau tenure = 0, maka TotalCharges biasanya kosong di dataset Telco ini.